# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [1]:
import os

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.5

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [2]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [3]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [4]:
BASE_PROMPT = """
You are a Financial Analyst. The client will ask you an investment question.
Do your best to help the client with their request based on your expertise.
Conclude with a succint and specific portfolio allocation tailored to the client's request.
"""

In [5]:
# Request from the client
request = "I'm 25 year old and have $1,000 saved. which Swiss stocks should I invest into?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [6]:
Markdown(response.content)

Investing in Swiss stocks can be a great way to diversify your portfolio, especially given Switzerland's stable economy and strong companies. Since you are 25 years old and have a relatively modest amount of $1,000 to invest, it's important to consider both growth potential and risk.

**Here are some Swiss stocks and sectors worth considering:**

1. **Nestlé (NESN)** - A global leader in food and beverages, Nestlé is known for its stability and dividend payments. It's a solid choice for a long-term investment.

2. **Novartis (NOVN)** - As a major player in the pharmaceuticals sector, Novartis has a strong pipeline of drugs and a history of stable growth.

3. **Roche (ROG)** - Another leading pharmaceutical company, Roche is known for its innovative treatments and strong market position.

4. **ABB (ABBN)** - A leader in technology and automation, ABB focuses on electrification and robotics, which are growing sectors.

5. **Swiss Re (SREN)** - As one of the world’s largest reinsurers, Swiss Re offers exposure to the insurance sector and can be a good hedge against market volatility.

**Portfolio Allocation:**

Given your age and the amount you have to invest, a balanced yet growth-oriented approach would be wise. Here's a suggested allocation:

- **Nestlé (NESN)**: 25% ($250)
- **Novartis (NOVN)**: 25% ($250)
- **Roche (ROG)**: 20% ($200)
- **ABB (ABBN)**: 15% ($150)
- **Swiss Re (SREN)**: 15% ($150)

This allocation provides a mix of stable and growth-oriented companies while diversifying across different sectors. Remember to consider transaction fees and the impact on your total investment when buying stocks. Additionally, it's wise to keep an eye on your investments and adjust your portfolio as needed over time.

## Simple Agent with Yahoo Finance News

We will now create a financial analyst bot again, but this time with an access to a tool, **Yahoo Finance News**. This tool allows the bot to retrieve the latest news about company stocks and provide more informed advice based on user requests.

The `YahooFinanceNewsTool` is part of the LangChain Community's available tool library: [LangChain tools](https://python.langchain.com/docs/integrations/tools/). 
For this workshop, we’ve made a simple modification to the `YahooFinanceNewsTool` and capable of retrieving a broader range of news articles. You can find the modified code in `src/yfinance_tool.py`

The `ChatOpenAI` class model includes a method, `bind_tools`, which simplifies the process of attaching and using tools with your bot.

In [7]:
from src.yfinance_tool import YahooFinanceNewsTool
from langchain_core.messages import ToolMessage

# Create a list of tools and a dictionnary of tool functions by name
tools = [YahooFinanceNewsTool()]
tools_by_name = {tool.name: tool for tool in tools}


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
user_question = (
    "How does Microsoft feels today comparing with NVIDIA?"
)
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you an investment question. Use the Yahoo Finance tool to look for news regarding specific stocks in the scope of the client's request.

Always convert company names to their exact ticker symbols using Yahoo ticker system (e.g., AAPL for Apple Inc., ZURN.SW for Zurich Insurance). Do not input full company names into the Yahoo Finance News tool.

Only use valid ticker symbols recognized on major stock exchanges (e.g., NASDAQ, NYSE, etc.). If a company does not have a valid ticker symbol, exclude it from analysis.


Based on your news research, identify which stocks are currently worth buying.

Finally, provide clear investment advice using the selected stocks. Do not assess risk.
"""

task_str = f"User question: {user_question}"

yahoo_finance_model = base_model.bind_tools(tools)

In [9]:
# Message list for the financial assistant model
messages = [
    SystemMessage(FINANCE_TOOL_PROMPT),
    HumanMessage(task_str),
]

# Invoke the financial assistant model with the messages
financial_analyst_output = yahoo_finance_model.invoke(messages)

# If the financial assistant model made tool calls, invoke the tool
if financial_analyst_output.tool_calls:
    news_list = []
    id_list = []
    for tool_call in financial_analyst_output.tool_calls:
        tool = tools_by_name[tool_call["name"]]
        news = tool.invoke(tool_call["args"])

        # display(Markdown(f"**Yahoo Finance news**: {news}"))

        news_list.append(news)
        id_list.append(tool_call["id"])

    # Combine the retrieved documents into a single string
    news_str = news
    # Message list with the retrieved documents for the base model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        HumanMessage(task_str),
        financial_analyst_output,
        *[
            ToolMessage(news_str, tool_call_id=tool_call_id)
            for (news_str, tool_call_id) in zip(news_list, id_list)
        ],
    ]

    # Invoke the base model with the messages
    financial_analyst_output_final = yahoo_finance_model.invoke(messages)

Markdown(financial_analyst_output_final.content)

yfinance.Ticker object <MSFT>
yfinance.Ticker object <NVDA>


Based on the latest news:

### Microsoft (MSFT)
- Microsoft is reportedly planning to lay off around 6,000 employees, which is approximately 3% of its global workforce. This could indicate a strategic shift or cost-cutting measures in response to market conditions.

### NVIDIA (NVDA)
- There were no specific news articles retrieved for NVIDIA in the current search, which may suggest that there are no major recent announcements impacting the company.

### Investment Advice
Given the current situation:
- **Microsoft (MSFT)** may not be the best buy at this moment due to the significant layoffs, which could reflect potential challenges ahead.
- **NVIDIA (NVDA)** appears stable without any recent negative news, but further analysis would be required to assess its current market position.

In conclusion, based on the available information, it may be prudent to consider NVIDIA (NVDA) for investment, while exercising caution with Microsoft (MSFT) due to the recent layoffs.

## Agentic system

We will now create a simple agentic system using **LangGraph**, consisting of four specialized agents.
This system enables more comprehensive financial analysis by dividing responsibilities among different agents:

- **Client Interface Agent (CIA)**: Rephrases the user's prompt to enhance the quality of responses from both the Financial Analyst and the Risk Advisor.

- **Financial Analyst Agent (FAA)**: The same financial agent as before, using **Yahoo Finance News**.

- **Risk Advisor Agent (RAA)**: Evaluates potential risks and provides cautionary advice regarding investments.

- **Sythetizer**: Combines the insights from both the Financial Analyst and the Risk Advisor into a single, coherent summary for the user.

To build our workflow with LangGraph, we need to implements functions that receive the current state and return state updates. These functions represent the nodes of the graph. Each node is assigned a label.

![Workflow](../imgs/workflow.png)

In [10]:
from typing import Literal

from langchain_core.messages import SystemMessage
from langgraph.graph import END
from langgraph.types import Command

## LangGraph Workflow and State

**LangGraph** helps us manage communication between our agents efficiently by defining a `State` class that will convey information from a node to the next during execution.

We keep our state simple by including only two attributes, but it's possible to include more:
- Messages: The ongoing conversation chain
- Analyses: Research findings from our agents

We use Python's dataclasses with special annotations (`Annotated`) to define how the state attributes should be updated throughout the workflow.

In [11]:
from dataclasses import dataclass, field
from typing import Annotated

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

In [12]:
@dataclass(kw_only=True)
class State:
    """Graph state for the financial analysis workflow."""

    messages: Annotated[list[BaseMessage], add_messages] = field(default_factory=list)
    financial_analysis: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )
    risk_analysis: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )

## The Agents in Detail

### Client Interface Agent (CIA)

The CIA serves as the interface between the user's request and the financial analysis workflow
- Evaluates client requests
- Reformulates the request for the financial analyst and risk advisor node

In [13]:
CLIENT_INTERFACE_PROMPT = """
You are a Client Interface Agent. Your role is to take the user's investment-related question
and reformulate it into a clear, structured, and precise prompt suitable for a Financial Analyst and Risk Advisor.

Do not ask clarifying or follow-up questions. Your output should be a single,
well-formed request ready for financial analysis.
"""

### Defining the Nodes and Flow

Let's now define our first graph Node. This section encompasses multiple **LangGraph** concepts at once and is worth spending some time on.

1. Defining the node:

    - To define our CIA node, we implement a function that receives the graph state and apply operations on it
    - In this case, the node calls the CIA model with structured output on the user request.

2. Defining the flow:

    - Last December, **LangGraph** released `Command`, a novel way of defining the graph edges directly within the nodes.
    - `Command` can return both state updates (update) and the next node (goto).
    - In the following example, the node updates the state messages with the CIA's response.
    - If the user request is out of scope, it terminates the workflow by going to the END node.
    - Otherwise, The generated response is passed down to the financial analyst and risk advisor nodes as part of the `state.messages`

In [14]:
# Define the CIA with the next node options
def client_interface_node(
    state: State,
) -> Command[Literal["financial_analyst", "risk_analyst", END]]:
    """CIA that generates a request for the financial analysis."""
    display(Markdown(f"**Client request received**: {state.messages[-1].content}"))

    # Message list for the orchestrator model
    messages = [
        SystemMessage(CLIENT_INTERFACE_PROMPT),
        *state.messages,
    ]

    # Invoke the CIA model
    orchestrator_output = base_model.invoke(messages)

    display(Markdown(f"**CIA Response:** {orchestrator_output.content}"))

    return Command(
        # Update the state messages with the CIA response
        update={"messages": orchestrator_output.content},
        # Go to worker nodes if the request is in scope, otherwise end the workflow
        goto=["financial_analyst", "risk_analyst"],
    )

### Financial Analyst Agent (FAA)

The FAA will give financial advice and will conduct news research:
- Receives a financial investment question from the CIA agent.
- Calls the `YahooFinanceNewsTool` to get current news on stocks linked to the client question.
- Formulates a clear investment advice based on the stocks with positive news of buying
- Sends the financial advice to the synthetizer

In [15]:
from langchain_core.messages import HumanMessage, ToolMessage

In [16]:
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you an investment question. Use the Yahoo Finance tool to look for news regarding specific stocks in the scope of the client's request.

Always convert company names to their exact ticker symbols using Yahoo ticker system (e.g., AAPL for Apple Inc., ZURN.SW for Zurich Insurance). Do not input full company names into the Yahoo Finance News tool.

Only use valid ticker symbols recognized on major stock exchanges (e.g., NASDAQ, NYSE, etc.). If a company does not have a valid ticker symbol, exclude it from analysis.

Discard any stocks with significant negative news.

Based on your news research, identify which stocks are currently worth buying.

Finally, provide clear investment advice using the selected stocks. Do not assess risk.
"""

# Create the financial assistant model from the base model with tool binding
fa_model = base_model.bind_tools(tools)

In [17]:
# Define the financial analyst node and the next node options
def financial_analyst_node(state: State) -> Command[Literal["synthesizer"]]:
    """Given an investment question, get financial advice using Yahoo Finance news."""
    display(Markdown("**Giving financial advice without risk information**"))
    # Message list for the financial assistant model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        *state.messages,
    ]

    # Invoke the financial assistant model with the messages
    fa_output = fa_model.invoke(messages)

    # If the financial assistant model made tool calls, invoke the tool
    if fa_output.tool_calls:
        news_list = []
        id_list = []
        for tool_call in fa_output.tool_calls:
            tool = tools_by_name[tool_call["name"]]
            news = tool.invoke(tool_call["args"])

            # display(Markdown(f"**Yahoo Finance news**: {news}"))

            news_list.append(news)
            id_list.append(tool_call["id"])

        # Message list with the retrieved documents for the base model
        messages = [
            SystemMessage(FINANCE_TOOL_PROMPT),
            *state.messages,
            fa_output,
            *[
                ToolMessage(news_str, tool_call_id=tool_call_id)
                for (news_str, tool_call_id) in zip(news_list, id_list)
            ],
        ]

        # Invoke the base model with the messages
        fa_output = fa_model.invoke(messages)

    # Update the state analyses with the financial analyst output content and go to the synthesizer node
    return Command(
        update={"financial_analysis": [fa_output.content]},
        goto="synthesizer",
    )

### Risk Analyst Agent (RAA)

The RAA will give a financial advice based on risk analysis. It offer a way to reduce investment's risk:
- Receive a financial investment question from the CIA agent.
- Give a financial advice
- Send it to the synthetizer

In [18]:
from langchain_core.messages import AIMessage

In [19]:
RISK_ANALYST_PROMPT = """
Evaluate the client's request strictly from a risk perspective.
Provide cautions, identify potential risks, and suggest ways to mitigate them.
Do not repeat or alter the Analyst's investment advice.
"""

In [20]:
# Define the risk analyst node and the next node options
def risk_analyst_node(state: State) -> Command[Literal[END]]:
    """Given the user prompt perform risk analysis."""
    display(Markdown("**Performing risk analysis.**"))

    # Access the previous responses
    finance_analysis = state.financial_analysis

    # Combine the previous messages into a single string
    complete_analyses = "The financial advisor's analysis:" + "\n\n---\n\n".join(
        finance_analysis
    )

    # Message list for the risk analyst
    messages = [
        SystemMessage(RISK_ANALYST_PROMPT),
        *state.messages,
        AIMessage(complete_analyses),
    ]

    # Invoke the base model with the messages
    risk_analyst_output = base_model.invoke(messages)

    # Update the state messages with the risk analyst's output content and go to the synthesizer node
    return Command(
        update={"risk_analysis": [risk_analyst_output.content]},
        goto="synthesizer",
    )

### Synthesiser

The Synthetiser is our final processing layer that:
- Collects the financial analyst's investment strategy
- Collects the risk advisor's investment 
- Creates a cohesive final report for the client

In [21]:
from langchain_core.messages import AIMessage

In [22]:
SYNTHESISER_PROMPT = """
You are a Financial Advisor.
Combine insights from both the Financial Analyst and Risk Advisor to generate clear, balanced investment advice.
Summarize key findings, address relevant risks, and provide a coherent investment strategy.
Conclude with a succint and specific portfolio allocation tailored to the client's request.
"""

In [23]:
# Define the synthesizer node and the next node options
def synthesizer_node(state: State) -> Command[Literal[END]]:
    """Synthesize full report from research analyses."""
    display(Markdown("**Synthesizing messages from the two analysts.**"))

    # Access the previous responses
    finance_analysis = state.financial_analysis

    risk_analysis = state.risk_analysis

    # Combine the research analyses into a single string
    financial_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in finance_analysis]
    )
    risk_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in risk_analysis]
    )

    # Message list for the RSA model
    messages = [
        SystemMessage(SYNTHESISER_PROMPT),
        AIMessage(financial_analysis_str),
        AIMessage(risk_analysis_str),
    ]

    # Invoke the base model with the messages
    synth_output = base_model.invoke(messages)

    # Update the state messages with the RSA output content and end the workflow
    return Command(
        update={"messages": synth_output},
        goto=END,
    )

## Building the Workflow Graph

Now that our nodes and communication flow are defined, we can build the graph!

In [24]:
from IPython.display import Image
from langgraph.graph import StateGraph

In [25]:
# Create a state graph builder
graph_builder = StateGraph(State)

# Define the entry point
graph_builder.set_entry_point("client_interface")

# Add the nodes
graph_builder.add_node("client_interface", client_interface_node)
graph_builder.add_node("financial_analyst", financial_analyst_node)
graph_builder.add_node("risk_analyst", risk_analyst_node)
graph_builder.add_node("synthesizer", synthesizer_node)

# The edges are defined by the commands !

# Compile the workflow
app = graph_builder.compile()

Let's visualize our Financial Analyst graph. Note that because the number of `"worker"` nodes is generated dynamically, it shows up as a single node in the image.

## Running the Workflow

Now that our workflow is built, let's test it! Once again, we can run it with `.invoke()`.

In [26]:
request = (
    "I'm 25 year old and have $1,000 saved. which Swiss stocks should I invest into?"
)

# Invoke the workflow with the client request
final_state = app.invoke({"messages": request})

**Client request received**: I'm 25 year old and have $1,000 saved. which Swiss stocks should I invest into?

**CIA Response:** Please provide a recommendation for Swiss stocks suitable for a 25-year-old investor with a $1,000 investment budget, considering factors such as growth potential, risk tolerance, and market trends as of October 2023.

**Giving financial advice without risk information**

**Performing risk analysis.**

yfinance.Ticker object <ZURN.SW>
yfinance.Ticker object <NOVN.SW>
yfinance.Ticker object <ROG.SW>
yfinance.Ticker object <UBSG.SW>
yfinance.Ticker object <CSGN.SW>


**Synthesizing messages from the two analysts.**

In [27]:
Markdown(final_state["messages"][-1].content)

### Combined Investment Advice

Based on insights from both financial analysis and risk assessment, here's a balanced investment strategy tailored to your request for investing in Swiss stocks with a budget of $1,000.

### Key Findings

1. **Strong Performers**:
   - **ZURN.SW (Zurich Insurance Group)**: Demonstrates robust growth with a 34% increase in net income, indicating strong market positioning.
   - **ROG.SW (Roche Holding)**: Actively expanding manufacturing capabilities and engaged in innovative drug trials, suggesting long-term growth potential.

2. **Cautionary Notes**:
   - **UBSG.SW (UBS Group)**: Facing reputational risks due to recent penalties related to Credit Suisse, which may affect its stock performance.
   - **CSGN.SW (Credit Suisse)**: Significant legal issues and recent negative news make it a poor investment choice.

3. **Market Risks**:
   - Volatility, currency fluctuations, concentration risks, and sector-specific downturns could impact your investments. Diversification and informed decision-making are crucial.

### Investment Strategy

To balance potential returns with associated risks, consider the following strategy:

1. **Diversified Allocation**:
   - Invest **60%** in **ZURN.SW (Zurich Insurance Group)**: Given its strong performance and growth outlook, this stock represents a solid investment opportunity.
   - Invest **40%** in **ROG.SW (Roche Holding)**: While slightly less aggressive than Zurich, Roche's focus on innovation and expansion makes it a compelling choice for long-term growth.

2. **Consider an ETF**: If you prefer a more diversified approach, consider investing in a Swiss-focused ETF that includes both Zurich and Roche, along with other stable companies. This can reduce individual stock risk while still offering exposure to the Swiss market.

3. **Regular Monitoring**: Stay updated on market trends and company news. Consider setting alerts for significant changes in stock performance or company announcements.

4. **Long-Term Perspective**: Aim for a long-term holding strategy to ride out market fluctuations. This approach often yields better returns compared to short-term trading.

### Suggested Portfolio Allocation

Given your budget of $1,000, here’s how you might allocate your investment:

- **ZURN.SW (Zurich Insurance Group)**: $600
- **ROG.SW (Roche Holding)**: $400

### Conclusion

This portfolio allocation focuses on strong, stable companies while addressing potential risks through diversification and a long-term investment strategy. Always consider your risk tolerance and investment goals, and consult with a financial advisor if needed. Regularly review your investments to ensure they align with your financial objectives.

## Conclusion
You've now learned how to build a sophisticated multi-agent system using LangGraph! This approach allows for:

- More complex and nuanced analysis
- Better division of responsibilities

Feel free to experiment with different agent configurations and workflow patterns to suit your specific needs.